# Backtest Functionality with testing variations using the new regression scorefile


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime as dt
import itertools as it
import ast

import sys
sys.path.append('../src')
from utils import *

In [5]:
# Import the price data from "../Raw Data/ETF_PriceFile_2017_to_2025.csv"
df_price_all = pd.read_csv('../Raw Data/ETF_PriceFile_2017_to_2025.csv', index_col=0)

In [6]:
# Import the old score summary from "../Score Summary/df_score_summary_old_reg.csv"
df_score_summary_new = pd.read_csv('../Score Summary/df_score_summary_new_reg.csv')

<h2>Run variations of Scorefiles generated (Fixed Income Included Universe) </h2>
<ul>
    <li>Days Offset range: 0 to 120 (increments of 10)</li>
    <li>Swap Threshold list: [0, 0.0025, 0.005, 0.01, 0.015]</li>
    <li>Trade Threshold: [0.015, 0.02, 0.025, 0.03, 0.0325, 0.035, 0.04, 0.05]</li>
</ul>

In [7]:
# Divide the score summary into training and testing sets

df_score_summary_train = df_score_summary_new[df_score_summary_new['Date'] < Excel_Date(pd.to_datetime('2024-01-01'))]
df_score_summary_test = df_score_summary_new[df_score_summary_new['Date'] >= Excel_Date(pd.to_datetime('2024-01-01'))]

In [ ]:
df_score_summary_train_copy = df_score_summary_train.copy()

df_return_summary = []
df_real_return_summary = []
for days_coefficient in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 150]:
    df_score_list = pd.DataFrame(df_score_summary_train_copy["Cheapness"] + days_coefficient * df_score_summary_train_copy["Combined Term 2"])
    df_score_list = df_score_list.rename(columns={0: "Score"})
    df_score_list["Date"] = df_score_summary_train_copy["Date"]
    df_score_list["Ticker"] = df_score_summary_train_copy["Ticker"]
    df_scorefile = df_score_list.pivot_table(index="Date", columns="Ticker", values="Score")
    date_range = df_scorefile.index
    for swap_threshold in [0, 0.005, 0.01, 0.015]:
        for trade_threshold in [0.015, 0.02, 0.03, 0.04, 0.05]:
            df_portfolio_backtest_train = Portfolio_Tracker(df_scorefile, df_price_all, date_range, 8, swap_threshold, trade_threshold, 1000, 0.0005)
            df_results = performance_summary(df_portfolio_backtest_train[["Return %", "Return % adjusted for swaps"]], 1, df_portfolio_backtest_train.index[0], date_range[-1])
            df_results.set_index("Group", inplace=True)
            df_return_summary.append({"Days Coefficient": days_coefficient,
                                      "Swap Threshold": swap_threshold,
                                      "Trade Threshold": trade_threshold,
                                      "Annualized Return %": df_results.loc["Return %", "Annualized Return"],
                                      "Standard Deviation": df_results.loc["Return %", "Standard Deviation"],
                                      "Downside Deviation": df_results.loc["Return %", "Downside Deviation"],
                                      "Max Drawdown": df_results.loc["Return %", "Max Drawdown"],})
            
            df_real_return_summary.append({"Days Coefficient": days_coefficient,
                                           "Swap Threshold": swap_threshold,
                                           "Trade Threshold": trade_threshold,
                                           "Annualized Return %": df_results.loc["Return % adjusted for swaps", "Annualized Return"],
                                           "Standard Deviation": df_results.loc["Return % adjusted for swaps", "Standard Deviation"],
                                           "Downside Deviation": df_results.loc["Return % adjusted for swaps", "Downside Deviation"],
                                           "Max Drawdown": df_results.loc["Return % adjusted for swaps", "Max Drawdown"],})
            
df_return_summary = pd.DataFrame(df_return_summary)
df_real_return_summary = pd.DataFrame(df_real_return_summary)

z:\Section-A\Friedenthal Financial\Research and Models\Tactical Trend\Tactical Model Charlie\ETF Model 2025\Organized Folder\notebooks\../src\utils.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_swaps["Existing Ticker"] = df_held_scores["Ticker"]
z:\Section-A\Friedenthal Financial\Research and Models\Tactical Trend\Tactical Model Charlie\ETF Model 2025\Organized Folder\notebooks\../src\utils.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_swaps["Current Score"] = df_held_scores[cu

TypeError: cannot do slice indexing on DatetimeIndex with these indexers [43097.0] of type float64

In [13]:
df_score_summary_train_copy = df_score_summary_train.copy()
df_score_list = pd.DataFrame(df_score_summary_train_copy["Cheapness"] + 32 * df_score_summary_train_copy["Combined Term 2"])


In [15]:
df_score_list["Date"] = df_score_summary_train_copy["Date"]
df_score_list["Ticker"] = df_score_summary_train_copy["Ticker"]


Ticker,ACWV US Equity,AGG US Equity,AMLP US Equity,ARKG US Equity,ARKK US Equity,ARKQ US Equity,ASHR US Equity,BBAX US Equity,BNDX US Equity,BOTZ US Equity,...,XLE US Equity,XLF US Equity,XLI US Equity,XLK US Equity,XLP US Equity,XLU US Equity,XLV US Equity,XLY US Equity,XME US Equity,XOP US Equity
Date,,,,,,,,,,,,,,,,,,,,,
43097.0,0.009363,0.001792,-0.008346,0.015538,0.045716,0.039068,0.020211,NaN,0.003265,0.032499,...,0.002709,0.016943,0.011323,0.020348,0.014707,0.011548,0.013062,0.013495,0.004231,-0.005924
43098.0,0.009379,0.001517,-0.005337,0.021569,0.052081,0.040709,0.017870,NaN,0.002781,0.036223,...,0.006292,0.021756,0.013348,0.023925,0.014185,0.008370,0.015480,0.018439,0.009397,0.000333
43102.0,0.009155,0.001923,-0.013897,0.010005,0.024447,0.027218,0.010277,NaN,0.003978,0.031157,...,0.002420,0.022443,0.011488,0.017554,0.016674,0.008782,0.009796,0.009529,0.000827,-0.005721
43103.0,0.007452,0.001870,-0.020840,0.008768,0.023660,0.018881,0.006366,NaN,0.003432,0.020835,...,-0.001944,0.018066,0.008372,0.012855,0.015214,0.008747,0.004933,0.007400,0.009463,-0.006319
43104.0,0.004598,0.002000,-0.020430,0.006978,0.023884,0.017351,0.006048,NaN,0.003227,0.014446,...,-0.001494,0.011810,0.004252,0.010983,0.012790,0.008462,0.004663,0.006606,0.011098,-0.004923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45282.0,0.008877,0.012276,0.016941,0.000994,0.046950,0.023330,-0.023099,-0.006492,0.008564,0.021165,...,-0.006260,0.020017,0.018270,0.017701,0.000558,0.006889,0.011499,0.022772,-0.000540,-0.006673
45286.0,0.009041,0.013519,0.015630,0.002042,0.047757,0.021569,-0.024531,-0.003763,0.010584,0.022743,...,-0.007651,0.021024,0.018882,0.018843,-0.000156,0.003621,0.012187,0.024028,0.002992,-0.009730
45287.0,0.007578,0.011604,0.015075,0.009510,0.046113,0.023626,-0.026344,-0.002054,0.008268,0.023723,...,-0.005895,0.019879,0.019598,0.020403,-0.001368,0.002509,0.011288,0.023091,0.006025,-0.006728
